In [1]:
import pandas_datareader.data as web #to collect data
import datetime as dt #to specify start and end dates

# import yfinance as yf

import eventstudy as es
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import scipy.stats as stats
from scipy.spatial.distance import cdist


from sklearn.neighbors import NearestNeighbors

import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.rolling import RollingOLS

from patsy import dmatrices
from tqdm.notebook import tqdm
tqdm.pandas()

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
import_folder_path = rf"..\..\[IN USE] Rookie Directors\[2] Firm Level Wrangling\Firm level Iterative\Firm Lev + Fin\City level\city_lev_output"
output_folder_path = "analysis_outputs"
pca_input_folder_path = rf"..\..\[IN USE] Rookie Directors\[1.5] Director Skills PCA\director_skills_pca"
supporting_folder_path = "supporting_datafiles"

In [3]:
firmLev0 = pd.read_pickle(rf"{import_folder_path}\Main_Firm_Fin_Loc_COMPLETE.pkl")

In [4]:
pca = pd.read_pickle(rf"{pca_input_folder_path}\Main_Director_COMPLETE_PCA.pkl")

In [5]:
firmlev_col = ["PC1_FactorScore_Standardised_mean"]

def FirmLevelCalc(frame):

    PC1_FactorScore_Standardised_mean = frame["PC1_FactorScore_Standardised"].mean()

    # ---------------------------------------------------------------------------------------------------------------------------------------------------

    output =    PC1_FactorScore_Standardised_mean

    return output

firmPCA = pca.groupby(['Symbol', 'AsOnDate']).progress_apply(FirmLevelCalc)

  0%|          | 0/28909 [00:00<?, ?it/s]

In [6]:
firmPCA2 = firmPCA.to_frame(name = "PC1_FactorScore_Standardised_mean").reset_index()
firmLev00 = firmLev0.merge(firmPCA2, on = ["AsOnDate", "Symbol"])

# PSM

## Verifying and removing those rows with no control data points

In [7]:
# ln_tobinq_longborrowincl2 = Dependent Variable

 

# Control variables


# percentwomendir
# percentbusydir
# lnboardsize
# percentindep 
# promoters_percent
# nonpromoterinstitutions_percent
# ln_marcap
# debttoequityratio
# ln_firmage
# pb
# hasdualitychairmanmd
# ln_rdtoassets
# pb

firmLev0.columns.to_list()
# Sample constraints ---> govtdummy==0 & findummy==0 & asonyear>2012

['FirmID',
 'Symbol',
 'AsOnDate',
 'TotDirCount',
 'AsOnYear',
 'LnBoardSize',
 'FemaleDirCount',
 'PercentWomenDir',
 'Women_dummy',
 'AvgAge',
 'CountRookie',
 'Rookie_dummy',
 'CountNonRookie',
 'CountIndep',
 'PercentIndep',
 'CountNonIndep',
 'CountRookieIndep',
 'CountRookieNonIndep',
 'CountNonRookieIndep',
 'CountNonRookieNonIndep',
 'CountPromoterDir',
 'CountBusyDir',
 'PercentBusyDir',
 'Busy_dummy',
 'HasCeoMD',
 'HasChairman',
 'HasPromoterOnBoard',
 'HasDualityChairmanMD',
 'IsFamilyManaged',
 'HasFamilyChairman',
 'HasFamilyChairmanAndCEO',
 'CountFirstTermDir',
 'CountFirstTermIndepDir',
 'PercentFirstTermIndepDir',
 'CountTurnOverIndepDir',
 'HasIndepTurnOver',
 'AvgTenureInYearsinCompTotal',
 'AvgTenureInYearsinCompIndep',
 'PercentRookieIndep',
 'PercentNonRookieIndep',
 'PercentRookieNonIndep',
 'PercentNonRookieNonIndep',
 'PercentRookie',
 'PercentNonRookie',
 'IsRookieBoard',
 'JustOneRookie',
 'TwoOrMoreRookies',
 'CountMBA',
 'PercentMBA',
 'CountIndepMBA',
 '

In [8]:
firmLev = firmLev00.copy()

firmLev["HasDualityChairmanMD"] = firmLev["HasDualityChairmanMD"].astype(int)

firmLev["NIC_1digit"] = firmLev["NIC code"].dropna().apply(lambda x: x[0:1])
firmLev["NIC_1digit"] = firmLev["NIC_1digit"].astype(float)

depVar = "TobinQ_longborrowincl2"


psmSample = firmLev.loc[ (firmLev["AsOnDate"] >= "2013-03-31")\
& (firmLev["govtdummy"] == 0) & (firmLev["findummy"] == 0) ].dropna(subset = depVar).copy()

psmSample["PercentRookieIndep50"] = np.where(psmSample["PercentRookieIndep"]>= 0.5, 1,0)
psmSample["PercentNonRookieIndep50"] = np.where(psmSample["PercentNonRookieIndep"]> 0.5, 1,0)

#psmSample["DummySum"] = psmSample["PercentRookieIndep50"] + psmSample["PercentNonRookieIndep50"]
psmSample["DummySumIndep"] = psmSample["PercentRookieIndep50"] + psmSample["PercentNonRookieIndep50"]

#psmSampleAll = psmSample.loc[ psmSample["DummySum"] == 1 ].reset_index(drop = True)

psmSampleIndep = psmSample.loc[ psmSample["DummySumIndep"] == 1 ].reset_index(drop = True).copy()



In [9]:
firmLev2 = firmLev.copy()
psmSampleIndep2 = psmSampleIndep.copy()

listCol = [
    "FirstYearPCodeList", "TwoYearPCodeList", "ThreeYearPCodeList", "PCodeList",
    "FirstYearIndepPCodeList", "TwoYearIndepPCodeList", "ThreeYearIndepPCodeList", "IndepPCodeList",
    "OtherFirstYearIndepPCode", "OtherTwoYearIndepPCode", "OtherThreeYearIndepPCode", "TotalIndepPCode",
    "OtherFirstYearPCode", "OtherTwoYearPCode", "OtherThreeYearPCode", "TotalPCode",
    "OtherFirstYearPCodeIndepExcl","OtherTwoYearPCodeIndepExcl", "OtherThreeYearPCodeIndepExcl", "TotalPCodeIndepExcl",
    "OtherFirstYearPCodeExcl", "OtherTwoYearPCodeExcl", "OtherThreeYearPCodeExcl", "TotalPCodeExcl"
]

firmLev2 = firmLev2.drop(listCol, axis = 1)
psmSampleIndep2 = psmSampleIndep2.drop(listCol, axis = 1)


firmLev2.to_csv(rf"{output_folder_path}\Main_Firm_PSM Ready_no filter v230525.csv")
psmSampleIndep2.to_csv(rf"{output_folder_path}\Main_Firm_PSM Ready_filter-Indep_gov_fin v230525.csv")


# # psmSampleAll --> 2101 rows 
# psmSampleIndep --> 1561 rows 

## PSM Functions

In [10]:
def LogitReg(sample, endog_var, exog_var):
    
    # Logit Regression
    endog = sample[[endog_var]]
    exog = sample[exog_var]
    exog = sm.add_constant(exog)
    
    log_reg = sm.Logit(endog, exog).fit()

    propensityScores = log_reg.predict(exog)
    
    return propensityScores

In [11]:
def MeanDiffTtest(sample, endog_var, exog_var, depVar, firmLev):

    firmLev = firmLev.rename( {depVar:f"{depVar}_2"}, axis = 1)

    colsAdd = []
    for i in range(-1, 4):
        if i != 0:
            colsAdd.append(f"AsOnYear_T+{i}")
            colsAdd.append(f"{depVar}T+{i}")
            if i>0 :
                colsAdd.append(depVar+f"(T+{i}) - (T-1)")

    newFrame= pd.DataFrame(columns = colsAdd, data = 0, index = sample.index, dtype = "int")
    sample = pd.concat([sample, newFrame], axis = 1)
    sample = sample.copy()
    
    for i in range(-1, 4):
        if i != 0:
            sample.loc[:, f"AsOnYear_T+{i}"] = sample["AsOnYear"] + i

    for i in range(-1, 4):
        if i != 0:
            sample.loc[:, f"{depVar}T+{i}"] = sample.merge(firmLev[["Symbol", "AsOnYear", f"{depVar}_2"]].copy(), left_on = ["Symbol", f"AsOnYear_T+{i}"],
                                                          right_on = ["Symbol", "AsOnYear"], how = "left")[f"{depVar}_2"]
    
    for i in range(1, 4):
        if i != 0:
            sample.loc[:, depVar+f"(T+{i}) - (T-1)"] = sample[f"{depVar}T+{i}"] - sample[f"{depVar}T+-1"]

    
    sample = sample.copy()
    
    group1 = sample.loc[ sample[endog_var] == 1].copy()
    group2 = sample.loc[ sample[endog_var] == 0].copy()
    
    t_stat, p_value = stats.ttest_ind(group1[depVar], group2[depVar], equal_var=False)  # Welch’s t-test (default)

    print("\n")
    print(depVar, ":")
    print("\n")
    print("T Statistic:", t_stat, " P Value:",p_value)
    print("Treated Mean:", group1[depVar].mean(), " Control Mean:", group2[depVar].mean(), " Diff:", group1[depVar].mean() - group2[depVar].mean())
    print("Treated N:", len(group1[depVar]), "; Control N:", len(group2[depVar]))
    print("[treated unique = ", len(group1.loc[ :, ["Symbol", "AsOnDate"]].drop_duplicates()), "]",\
          "[control unique = ", len(group2.loc[ :, ["Symbol", "AsOnDate"]].drop_duplicates()), "]"
         )
    print("\n")

    print("━"*120)
    print(f'{"Matching Variable":<40} {"Treatment Firms":<20} {"Control Firms":<20} {"Test of Diff (p value)":<20}')
    print(f'{" ":<40} {"N = " + str(len(group1[depVar])):<20} {"N = " + str(len(group2[depVar])):<20}')
    print("-"*120)

    for var in exog_var:
        treatMean = group1[var].mean()
        controlMean = group2[var].mean()
        p_value = stats.ttest_ind(group1[var], group2[var], equal_var=False)[1]
        print(f'{var:<40} {treatMean:<20.4f} {controlMean:<20.4f} {p_value:<20.4f}')

    print("━"*120, "\n")
    # print(depVar, " across years:\n")

    # for i in range(1,4):
    #     sample = sample.dropna(subset = [depVar+f'(T+{i}) - (T-1)'])
        
    # group1 = sample.loc[ sample[endog_var] == 1].copy()
    # group2 = sample.loc[ sample[endog_var] == 0].copy()

    # print("━"*150, "\n")
    # print(f'{depVar:<40}{" ":<20}{"Treatment Firms":<20}{"Control Firms":<20}{"Difference":<20}{"Test of Diff":<20}{"Test of Diff"}')
    # print(f'{" ":<120}{"(t stat)":<20}{"(p value)":<20}')

    # print("─"*150, "\n")


    # for i in range(1,4):
    #     t_stat2, p_value2 = stats.ttest_ind(group1[depVar+f'(T+{i}) - (T-1)'], group2[depVar+f'(T+{i}) - (T-1)'], equal_var=False)  # Welch’s t-test (default)
        
    #     treatedMean = group1[depVar+f'(T+{i}) - (T-1)'].mean()
    #     controlMean = group2[depVar+f'(T+{i}) - (T-1)'].mean()
    #     diffMean = treatedMean - controlMean

    #     treatedMedian = group1[depVar+f'(T+{i}) - (T-1)'].median()
    #     controlMedian = group2[depVar+f'(T+{i}) - (T-1)'].median()
    #     diffMedian = treatedMedian - controlMedian

    #     print(f'{"Year_T+" + str(i) +" - Year_T-1":<40}{"<MEAN>":<20}{treatedMean:<20.4f}{controlMean:<20.4f}{diffMean:<20.4f}{t_stat2:<20.4f}{p_value2:<20.4f}')

    #     label1 = "Treated N: " + str(len(group1[depVar+f'(T+{i}) - (T-1)']))
    #     label2 = "Control N: " + str(len(group1[depVar+f'(T+{i}) - (T-1)']))
        
    #     print(f'{label1 + " "*5 + label2:<40}{"<MEDIAN>":<20}{treatedMedian:<20.4f}{controlMedian:<20.4f}{diffMedian:<20.4f}')
        
    #     print("-"*150, "\n")
        
    # print("━"*150, "\n")

    # return

In [12]:
def PsmReplac(sample, endog_var, exog_var, depVar, firmLev):

    # Logit Regression
    sample["propensityScore"] = LogitReg(sample, endog_var, exog_var)

    treated = sample.loc[ sample[endog_var] == 1].copy()
    control = sample.loc[ sample[endog_var] == 0].copy()

    # Nearest Neighbours
    nn = NearestNeighbors(n_neighbors = 1, metric = "euclidean")
    nn.fit(control[["propensityScore"]])

    distances, indices = nn.kneighbors(treated[["propensityScore"]])
    
    matchedControl = control.iloc[indices.flatten()].copy()
    
    matched = pd.concat([treated, matchedControl])
    matched.reset_index(drop=True, inplace=True)

    MeanDiffTtest(matched, endog_var, exog_var, depVar, firmLev)

    return

In [13]:
# Func PSM non replacement
def PsmNonReplac(sample, endog_var, exog_var, depVar, firmLev):

    # Logit Regression
    sample["propensityScore"] = LogitReg(sample, endog_var, exog_var)

    # Separate treated and control groups
    treated = sample[sample[endog_var] == 1].copy()
    control = sample[sample[endog_var] == 0].copy()
    
    # Compute pairwise distances (absolute difference in propensity scores)
    dist_matrix = cdist(treated[['propensityScore']], control[['propensityScore']], metric='euclidean')
    
    # Match without replacement
    treated_indices = []
    matched_indices = []
    used_control_indices = set()
    
    for i in range(len(treated)):
        if len(used_control_indices) >= len(control):  # Stop if no controls left
            print("Warning: Not enough control units to match all treated units.")
            break
        
        # Get nearest control unit index that hasn't been used
        match_idx = np.argmin(dist_matrix[i])
        
        while match_idx in used_control_indices:  # Ensure it's not already matched
            dist_matrix[i, match_idx] = np.inf  # Temporarily set distance to infinity

            if np.all(dist_matrix[i] == np.inf):  # If all controls are exhausted
                print(f"No available control for treated unit {i}, skipping.")
                match_idx = None
                break
            
            match_idx = np.argmin(dist_matrix[i])
        
        used_control_indices.add(match_idx)
        matched_indices.append(match_idx)
        treated_indices.append(i)
    
    # Retrieve matched units
    matched_control = control.iloc[matched_indices].copy()
    matched_treated = treated.iloc[treated_indices].copy()
    
    # Combine matched treated and control units
    matched_data = pd.concat([matched_treated.reset_index(drop=True), matched_control.reset_index(drop=True)])
    
    # Reset index
    matched_data.reset_index(drop=True, inplace=True)


    # Mean difference and T Test
    MeanDiffTtest(matched_data, endog_var, exog_var, depVar, firmLev)

    return
    


In [14]:
def prep_sample(sample, controlVars, yearDummy = 0, industryDummy = 0) -> tuple[pd.DataFrame, list[str]]:
    
    sample = sample.replace([np.inf, -np.inf], pd.NA).dropna(subset = controlVars).reset_index(drop=True)

    if yearDummy == 1:
        yearDummies = pd.get_dummies( sample["AsOnYear"], prefix = "YearDummy_", drop_first = True, dtype = "int")
        sample = pd.concat([sample, yearDummies], axis = 1)
    else:
        yearDummies = pd.DataFrame()
    
    if industryDummy == 1:
        industryDummies = pd.get_dummies( sample["NIC_1digit"], prefix = "IndustryDummy_", drop_first = True, dtype = "int")
        sample = pd.concat([sample, industryDummies], axis = 1)
    else:
        industryDummies = pd.DataFrame()

    exog_var = controlVars + yearDummies.columns.to_list() + industryDummies.columns.to_list()
    
    return sample, exog_var

# PSM on firm level output + following controls

In [15]:
controlVars = ["PercentWomenDir", "PercentBusyDir",
               "LnBoardSize", "PercentIndep",
               "Promoters_percent", "NonpromoterInstitutions_percent",
               "ln_marcap", "ln_rdtoassets", "Debt to equity ratio",
               "HasDualityChairmanMD"]

# Version 0: No filtration

In [16]:
# psmSampleIndep0 = psmSampleIndep.copy()

# yearDummies0 = pd.get_dummies( psmSampleIndep0["AsOnYear"], prefix = "YearDummy_", drop_first = True, dtype = "int")
# psmSampleIndepWithYearDummies0 = pd.concat([psmSampleIndep0, yearDummies0], axis = 1)

# industryDummies0 = pd.get_dummies( psmSampleIndep0["NIC_2digit"], prefix = "IndustryDummy_", drop_first = True, dtype = "int")
# psmSampleIndepWithYearIndustryDummies0 = pd.concat([psmSampleIndepWithYearDummies0, industryDummies0], axis = 1)

## No fixed effects

### PSM with replacement

In [17]:
sample, exog_var = prep_sample(psmSampleIndep, controlVars, 0, 0)
PsmReplac(sample, "PercentRookieIndep50", exog_var, "TobinQ_longborrowincl2", firmLev)

Optimization terminated successfully.
         Current function value: 0.309942
         Iterations 8


TobinQ_longborrowincl2 :


T Statistic: 4.385453592661363  P Value: 1.3646808634343538e-05
Treated Mean: 2.2746347949165813  Control Mean: 1.4650833950089122  Diff: 0.8095513999076691
Treated N: 362 ; Control N: 362
[treated unique =  362 ] [control unique =  162 ]


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Matching Variable                        Treatment Firms      Control Firms        Test of Diff (p value)
                                         N = 362              N = 362             
------------------------------------------------------------------------------------------------------------------------
PercentWomenDir                          0.1726               0.2130               0.0000              
PercentBusyDir                           0.1804               0.1350               0.0196    

## Using Year fixed effects

### PSM with replacement

In [18]:
sample, exog_var = prep_sample(psmSampleIndep, controlVars, 1, 0)
PsmReplac(sample, "PercentRookieIndep50", exog_var, "TobinQ_longborrowincl2", firmLev)

Optimization terminated successfully.
         Current function value: 0.302483
         Iterations 8


TobinQ_longborrowincl2 :


T Statistic: 3.1424582783829873  P Value: 0.001751238317470431
Treated Mean: 2.2746347949165813  Control Mean: 1.4273346066031307  Diff: 0.8473001883134506
Treated N: 362 ; Control N: 362
[treated unique =  362 ] [control unique =  161 ]


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Matching Variable                        Treatment Firms      Control Firms        Test of Diff (p value)
                                         N = 362              N = 362             
------------------------------------------------------------------------------------------------------------------------
PercentWomenDir                          0.1726               0.2158               0.0000              
PercentBusyDir                           0.1804               0.1296               0.0085     

## Using Year + Industry fixed effects

### PSM with replacement

In [19]:
sample, exog_var = prep_sample(psmSampleIndep, controlVars, 1, 1)
PsmReplac(sample, "PercentRookieIndep50", exog_var, "TobinQ_longborrowincl2", firmLev)

Optimization terminated successfully.
         Current function value: 0.300492
         Iterations 8


TobinQ_longborrowincl2 :


T Statistic: 4.153637347197803  P Value: 3.79049032654961e-05
Treated Mean: 2.2746347949165813  Control Mean: 1.537246239589754  Diff: 0.7373885553268273
Treated N: 362 ; Control N: 362
[treated unique =  362 ] [control unique =  155 ]


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Matching Variable                        Treatment Firms      Control Firms        Test of Diff (p value)
                                         N = 362              N = 362             
------------------------------------------------------------------------------------------------------------------------
PercentWomenDir                          0.1726               0.2048               0.0000              
PercentBusyDir                           0.1804               0.1155               0.0005       

# Version 1: promoterholding25==1 & hasfamilychairmanandceo==1

In [20]:
psmSampleIndep1 = psmSampleIndep.loc[ (psmSampleIndep["promoterholding25"]==1) & (psmSampleIndep["HasFamilyChairmanAndCEO"]==1) ].reset_index(drop = True).copy()

## No fixed effects

### PSM with replacement

In [21]:
sample, exog_var = prep_sample(psmSampleIndep1, controlVars, 0, 0)
PsmReplac(sample, "PercentRookieIndep50", exog_var, "TobinQ_longborrowincl2", firmLev)

Optimization terminated successfully.
         Current function value: 0.341259
         Iterations 8


TobinQ_longborrowincl2 :


T Statistic: 0.23352173245786953  P Value: 0.8155880161709099
Treated Mean: 2.337953219127978  Control Mean: 2.20659987851185  Diff: 0.13135334061612802
Treated N: 153 ; Control N: 153
[treated unique =  153 ] [control unique =  73 ]


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Matching Variable                        Treatment Firms      Control Firms        Test of Diff (p value)
                                         N = 153              N = 153             
------------------------------------------------------------------------------------------------------------------------
PercentWomenDir                          0.1753               0.2024               0.0086              
PercentBusyDir                           0.1556               0.1337               0.4276         

C:\Users\SHIVAM\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


## Using Year fixed effects

### PSM with replacement

In [22]:
sample, exog_var = prep_sample(psmSampleIndep1, controlVars, 1, 0)
PsmReplac(sample, "PercentRookieIndep50", exog_var, "TobinQ_longborrowincl2", firmLev)

Optimization terminated successfully.
         Current function value: 0.327899
         Iterations 8


TobinQ_longborrowincl2 :


T Statistic: 1.1147267359991049  P Value: 0.2659072497729712
Treated Mean: 2.337953219127978  Control Mean: 1.9315728153534106  Diff: 0.4063804037745675
Treated N: 153 ; Control N: 153
[treated unique =  153 ] [control unique =  79 ]


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Matching Variable                        Treatment Firms      Control Firms        Test of Diff (p value)
                                         N = 153              N = 153             
------------------------------------------------------------------------------------------------------------------------
PercentWomenDir                          0.1753               0.1962               0.0398              
PercentBusyDir                           0.1556               0.1211               0.1856         

C:\Users\SHIVAM\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:523: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


## Using Year + Industry fixed effects

### PSM with replacement

In [23]:
sample, exog_var = prep_sample(psmSampleIndep1, controlVars, 1, 1)
PsmReplac(sample, "PercentRookieIndep50", exog_var, "TobinQ_longborrowincl2", firmLev)

         Current function value: inf
         Iterations: 35


C:\Users\SHIVAM\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2383: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\Users\SHIVAM\anaconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2441: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q * linpred)))


LinAlgError: Singular matrix

# Version 2: promoterholding25==1 & hasfamilychairmanandceo==0

In [24]:
psmSampleIndep2 = psmSampleIndep.loc[ (psmSampleIndep["promoterholding25"]==1) & (psmSampleIndep["HasFamilyChairmanAndCEO"]==0) ].reset_index(drop = True).copy()

## No fixed effects

### PSM with replacement

In [25]:
sample, exog_var = prep_sample(psmSampleIndep2, controlVars, 0, 0)
PsmReplac(sample, "PercentRookieIndep50", exog_var, "TobinQ_longborrowincl2", firmLev)

Optimization terminated successfully.
         Current function value: 0.272243
         Iterations 8


TobinQ_longborrowincl2 :


T Statistic: 5.952333164271543  P Value: 8.800947853755666e-09
Treated Mean: 2.321013543752278  Control Mean: 0.8629280430044532  Diff: 1.4580855007478248
Treated N: 197 ; Control N: 197
[treated unique =  197 ] [control unique =  71 ]


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Matching Variable                        Treatment Firms      Control Firms        Test of Diff (p value)
                                         N = 197              N = 197             
------------------------------------------------------------------------------------------------------------------------
PercentWomenDir                          0.1726               0.1990               0.0040              
PercentBusyDir                           0.1984               0.1120               0.0013       

## Using Year fixed effects

### PSM with replacement

In [26]:
sample, exog_var = prep_sample(psmSampleIndep2, controlVars, 1, 0)
PsmReplac(sample, "PercentRookieIndep50", exog_var, "TobinQ_longborrowincl2", firmLev)

Optimization terminated successfully.
         Current function value: 0.265209
         Iterations 8


TobinQ_longborrowincl2 :


T Statistic: 5.882040714474905  P Value: 1.361010439908205e-08
Treated Mean: 2.321013543752278  Control Mean: 0.9035668122563307  Diff: 1.4174467314959474
Treated N: 197 ; Control N: 197
[treated unique =  197 ] [control unique =  77 ]


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Matching Variable                        Treatment Firms      Control Firms        Test of Diff (p value)
                                         N = 197              N = 197             
------------------------------------------------------------------------------------------------------------------------
PercentWomenDir                          0.1726               0.2359               0.0000              
PercentBusyDir                           0.1984               0.1254               0.0070       

## Using Year + Industry fixed effects

### PSM with replacement

In [27]:
sample, exog_var = prep_sample(psmSampleIndep2, controlVars, 1, 1)
PsmReplac(sample, "PercentRookieIndep50", exog_var, "TobinQ_longborrowincl2", firmLev)

         Current function value: 0.258308
         Iterations: 35


TobinQ_longborrowincl2 :


T Statistic: 5.501792778200646  P Value: 8.839891856974345e-08
Treated Mean: 2.321013543752278  Control Mean: 0.9518884961807812  Diff: 1.369125047571497
Treated N: 197 ; Control N: 197
[treated unique =  197 ] [control unique =  69 ]


━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Matching Variable                        Treatment Firms      Control Firms        Test of Diff (p value)
                                         N = 197              N = 197             
------------------------------------------------------------------------------------------------------------------------
PercentWomenDir                          0.1726               0.2308               0.0000              
PercentBusyDir                           0.1984               0.1317               0.0152              
LnBoardSize                  

C:\Users\SHIVAM\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


YearDummy__2018                          0.0508               0.0254               0.1891              
YearDummy__2019                          0.0508               0.0152               0.0487              
YearDummy__2020                          0.1066               0.5787               0.0000              
YearDummy__2021                          0.1015               0.0254               0.0019              
YearDummy__2022                          0.1218               0.0355               0.0014              
YearDummy__2023                          0.1168               0.0508               0.0181              
YearDummy__2024                          0.0711               0.0508               0.4008              
IndustryDummy__1.0                       0.1168               0.0609               0.0517              
IndustryDummy__2.0                       0.7665               0.8325               0.1024              
IndustryDummy__3.0                       0.0102               0.